# Overview
For Pseudo Labellings<br>
 1. hasMask==1<br>
  -> average the mask-existing submission files by pixel<br>
 2. hasMask==0<br>
  -> Make score = NaN<br>
 3. hasMask==unlabelled<br>
  -> average all the submission files by pixel<br>
  <br>
  - Version101<br>
  Erased Pixel Num Threshold Removal from Pseudo==1 Ensembles<br>
  For (Fish,Flower,Gravel,Sugar)==(NaN,NaN,NaN,NaN)<br>
   - if pseudo==1 availabe:<br>
   -> GASSAN ensemble<br>
   - elif pseudo==NaN exists:<br>
   -> Choose the most Frequently notna() Cloud Type 'ALONE', to conduct a positive ensemble<br>

In [1]:
import os
import numpy as np
import pandas as pd
import cv2, time, random, warnings
from tqdm import tqdm, tqdm_notebook

warnings.filterwarnings('ignore')

In [2]:
VERSION = 'PseudoPixelEnsemble_101'
PATH = '../input'
SUBMISSION_PATH = '../input/submissions'
PSEUDO_VERSION = '7'
REMOVE_THRESHOLD = 5000

# 1. Show All Submission FIles & State the Files to Use

In [3]:
os.listdir(SUBMISSION_PATH)

['submission_ResBase001.csv',
 'submission_segmentation_and_classifier.csv',
 'submission_Heng002_0.csv',
 'submission_UNetEffb2_004_0.csv',
 'submission_DSUNet106_0.csv',
 'submission_UNetEffb2_004_9.csv',
 'submission_Heng007_7.csv',
 'submission_21model_keita.csv',
 'submission_DSUNet104_0.csv',
 'submission_Heng004_0.csv',
 'submission_PseudoPixelEnsemble_101.csv',
 'submission_UNetEffb2_004_12.csv',
 'submission_UNetEffb2_006_0.csv',
 'submission_pseudoPixelEnsemble_001.csv',
 'submission_pseudo_001.csv',
 'submission_PseudoPixelEnsemble_002.csv',
 'submission_UNetEffb2_003_0.csv',
 'submission_DSUNet103_0.csv',
 'submission_Yirun_002_0.csv',
 'submission_UNetEffb3_001_0.csv',
 'submission_Heng006_0.csv',
 'submission_UNetEffb2_008_0.csv',
 'submission_UNetEffb2_005_0.csv',
 'keita_ensemble_files',
 'submission_UNetEffb2_007_0.csv',
 'submission_Yirun_001_0.csv',
 'submission_Heng003_0.csv',
 'submission_UNetEffb2_002_0.csv',
 'submission_Heng007_6.csv',
 '.ipynb_checkpoints',
 's

In [4]:
os.listdir(SUBMISSION_PATH+'/keita_ensemble_files')

['submission_eff-b4(CosineAnnealingLR)_0.6488.csv',
 'submission_FPN(efficientb2)-ver1.0_0.6448.csv',
 'submission_model_ensemble_0.6564.csv',
 'submission_incepv2_0.658.csv',
 'submission_effb2_0.658.csv',
 'submission_FPN(efficientb2)-ver2_0.6463.csv',
 'submission_DSUNet105_0_0.6480.csv',
 'submission_Unet(efficientnet-b2)-ver1_0.6552.csv',
 'submission_FPN(efficientb2)_0.6548.csv',
 'submission_resnet101_attention_0.6513.csv',
 'submission_unet_res101_0.6553.csv',
 'submission_someensemble_0.6511.csv',
 'submission_pixcel_0.6601.csv',
 'submission_Unet(seresnet101)_0.6486.csv',
 'submission_3Fold_0.6527.csv',
 'submission_Unet(se_resnext50)_0.6513.csv',
 'submission_UNetEffb2_004_0.6555.csv',
 'submission_19model_ensemble_0.6624.csv',
 'submission_efficientb5_0.6520.csv',
 'submission_FPN(efficientb2)_0.6583.csv',
 'submission_PSPNet(ResNet101)_0.6358.csv',
 'submission_Linknet(efficientb2_AWS)_ver2_0.6490.csv']

In [5]:
use_files = [
    '/submission_UNetEffb2_004_0.csv',
    '/submission_21model_keita.csv',
    '/submission_Heng005_0.csv',
    '/submission_DSUNet105_0.csv',
    '/keita_ensemble_files/submission_eff-b4(CosineAnnealingLR)_0.6488.csv',
    '/keita_ensemble_files/submission_model_ensemble_0.6564.csv',
    '/keita_ensemble_files/submission_incepv2_0.658.csv',
    '/keita_ensemble_files/submission_pixcel_0.6601.csv',
    '/keita_ensemble_files/submission_effb2_0.658.csv',
    '/keita_ensemble_files/submission_resnet101_attention_0.6513.csv',
    '/keita_ensemble_files/submission_19model_ensemble_0.6624.csv',
    '/keita_ensemble_files/submission_FPN(efficientb2)_0.6583.csv',
    '/keita_ensemble_files/submission_Linknet(efficientb2_AWS)_ver2_0.6490.csv',
]

# 2. Functions

In [6]:
def rle_decode(mask_rle: str = '', shape: tuple = (350, 525)):
    '''
    Decode rle encoded mask.
    
    :param mask_rle: run-length as string formatted (start length)
    :param shape: (height, width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0] * shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape, order='F')

def make_mask(label):
    """
    Create mask based on df, image name and shape.
    """
    masks = np.zeros((350, 525), dtype=np.float32)
    if label is not np.nan:
        mask = rle_decode(label)
        masks[:, :] = mask
            
    return masks

def mask2rle(img):
    """
    Convert mask to rle.
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    """
    pixels = img.T.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return " ".join(str(x) for x in runs)

# 3. Pseudo Ensemble

## 3.1 Preparation

In [7]:
# Sample Submission
sub_df = pd.read_csv('../input/sample_submission.csv')
sub_df.head()

,Image_Label,EncodedPixels
0,002f507.jpg_Fish,1 1
1,002f507.jpg_Flower,1 1
2,002f507.jpg_Gravel,1 1
3,002f507.jpg_Sugar,1 1
4,0035ae9.jpg_Fish,1 1


In [8]:
# read all submission csv files
submission_df_list = []
for file in tqdm(use_files):
    submission_df_list.append(pd.read_csv(SUBMISSION_PATH+file))
    

100%|██████████| 13/13 [00:03<00:00,  3.53it/s]


In [9]:
# read the Pseudo Label Files
pseudo = sub_df[['Image_Label']].copy()
for cloud_type in ['Fish', 'Flower', 'Gravel', 'Sugar']:
    df = pd.read_csv(PATH + f'/pseudo/pseudo_{cloud_type}_ClassEffb0_{cloud_type}-{PSEUDO_VERSION}.csv')
    df = df[df['from']=='test'].reset_index(drop=True)
    
    pseudo_image_labels = df[df['hasMask']==0]['path'].apply(lambda x: x.split('/')[-1] + '_' + cloud_type).to_numpy()
    for im_lb in tqdm(pseudo_image_labels):
        pseudo.loc[pseudo['Image_Label']==im_lb, 'flag'] = 0
    
    pseudo_image_labels = df[df['hasMask']==1]['path'].apply(lambda x: x.split('/')[-1] + '_' + cloud_type).to_numpy()
    for im_lb in tqdm(pseudo_image_labels):
        pseudo.loc[pseudo['Image_Label']==im_lb, 'flag'] = 1
        pseudo_df = pd.DataFrame(pseudo_image_labels, columns=['Image_Label'])

pseudo.head()

100%|██████████| 1920/1920 [00:07<00:00, 247.40it/s]


,Image_Label,flag
0,002f507.jpg_Fish,0.0
1,002f507.jpg_Flower,0.0
2,002f507.jpg_Gravel,1.0
3,002f507.jpg_Sugar,0.0
4,0035ae9.jpg_Fish,NaN


## 3.2 Ensemble Starts Here

In [10]:
# Ensemble Starts Here

submission = sub_df.copy()

# if hasMask == 1.0
def pseudo_positive_ensemble(encoded_pixel_list):
    mask_list = []
    for ep in encoded_pixel_list:
        if ep==ep:
            mask_list.append(make_mask(ep))
    if len(mask_list) != 0:
        mask = np.round(np.average(mask_list, axis=0))
    else:
        return np.nan
    if mask.sum()!=0:
        return mask2rle(mask)
    else:
        return np.nan

# if hasMask == NaN
def pseudo_nullified_ensemble(encoded_pixel_list):
    mask_list = []
    for ep in encoded_pixel_list:
        mask_list.append(make_mask(ep))
    if len(mask_list) != 0:
        mask = np.round(np.average(mask_list, axis=0))
        if np.count_nonzero(mask == 1) < REMOVE_THRESHOLD:
            return np.nan
    else:
        return np.nan
    if mask.sum()!=0:
        return mask2rle(mask)
    else:
        return np.nan


##############################################
# Ensemble with Pseudo Classification Labels #
##############################################

for im_lb in tqdm_notebook(submission['Image_Label'].to_numpy()):
    
    # make a list of EncodedPixels : String of each Submission FIles
    encoded_pixel_list = []
    for df in submission_df_list:
        encoded_pixel_list.append(df[df['Image_Label']==im_lb]['EncodedPixels'].item())
    
    # Check the Pseudo Label for the certain Image_Label and work with Ensembling
    pseudo_label = pseudo[pseudo['Image_Label']==im_lb]['flag'].item()
    
    if pseudo_label!=pseudo_label:# if NaN
        pred = pseudo_nullified_ensemble(encoded_pixel_list)

    elif pseudo_label==1.0:
        pred = pseudo_positive_ensemble(encoded_pixel_list)
        
    elif pseudo_label==0.0:
        pred = np.nan
        
    else:
        print('Unknown Pseudo Label')
        
    # Submission CSV Maker
    submission.loc[submission['Image_Label']==im_lb, 'EncodedPixels'] = pred
    

In [11]:
##########################################################
# Cover up (Fish,Flower,Gravel,Sugar)==(NaN,NaN,NaN,NaN) #
##########################################################

cloud_dict = {0:'Fish', 1:'Flower', 2:'Gravel', 3:'Sugar'}

# GASSAN
def pseudo_GASSAN_ensemble(encoded_pixel_list):
    mask_list = []
    for ep in encoded_pixel_list:
        if ep==ep:
            mask_list.append(make_mask(ep))
    if len(mask_list) != 0:
        mask = np.round(np.maximum(mask_list, axis=0)) # Taking the Maximums : Force all 1 to 1
    else:
        return np.nan
    if mask.sum()!=0:
        return mask2rle(mask)
    else:
        return np.nan
    
sub_copy = submission.copy()
sub_copy['ImageID'] = sub_copy['Image_Label'].apply(lambda x: x.split('_')[0])
pseudo['ImageID'] = pseudo['Image_Label'].apply(lambda x: x.split('_')[0])

# Find out (Fish,Flower,Gravel,Sugar)==(NaN,NaN,NaN,NaN) ImageIDs
cover_up_id_list = []
for image_id in tqdm(sub_copy['ImageID'].unique()):
    if sub_copy[sub_copy['ImageID']==image_id]['EncodedPixels'].count() == 0:
        cover_up_id_list.append(image_id)
        
# Cover Up (Fish,Flower,Gravel,Sugar)==(NaN,NaN,NaN,NaN) ImageIDs
for image_id in tqdm(cover_up_id_list):
    
    nan_type_list = []
    nan_EncodingPixel_list = []
    flag_one = 0

    for i in range(4):

        ps = pseudo[pseudo['ImageID']==image_id]['flag'].tolist()[i]

        if ps == 1:
            flag_one = 1 # if flag_one==1: skip the pseudo==NaN part
            encoded_pixel_list = []
            for df in submission_df_list:
                encoded_pixel_list.append(df[df['Image_Label']==image_id+'_'+cloud_dict[i]]['EncodedPixels'].item())
            ################### GASSAN ensemble for pseudo == 1
            pred = pseudo_GASSAN_ensemble(encoded_pixel_list)
            ###################
            sub_copy.loc[sub_copy['Image_Label']==image_id+'_'+cloud_dict[i], 'EncodedPixels'] = pred

        elif ps != ps: # if NaN
            nan_type_list.append(cloud_dict[i])
            encoded_pixel_list = []
            for df in submission_df_list:
                encoded_pixel_list.append(df[df['Image_Label']==im_lb]['EncodedPixels'].item())
            nan_EncodingPixel_list.append(encoded_pixel_list)

    if (flag_one == 0) & (len(nan_type_list) != 0):
        count_list = []
        count = 0
        for i, eplist in enumerate(nan_EncodingPixel_list):
            count_list.append(pd.Series(eplist).count())
            if pd.Series(eplist).count() > count:
                best_idx = i

        if (np.array(count_list)==max(count_list)).sum() == 1:
            ct = nan_type_list[best_idx] # Cloud Type which has the most notna() EncodedPixels 'ALONE'
            encoded_pixel_list = []
            for df in submission_df_list:
                encoded_pixel_list.append(df[df['Image_Label']==image_id+'_'+ct]['EncodedPixels'].item())
            ################### positive ensemble for pseudo == NaN & top count notna() Cloud Type
            pred = pseudo_positive_ensemble(encoded_pixel_list)
            ###################
            sub_copy.loc[sub_copy['Image_Label']==image_id+'_'+cloud_dict[i], 'EncodedPixels'] = pred

submission = sub_copy[['Image_Label', 'EncodedPixels']]

100%|██████████| 58/58 [00:04<00:00, 12.54it/s]


# 4. Submission

In [12]:
submission.to_csv(SUBMISSION_PATH+f'/submission_{VERSION}.csv', index=False)
submission.head(10)

,Image_Label,EncodedPixels
0,002f507.jpg_Fish,NaN
1,002f507.jpg_Flower,NaN
2,002f507.jpg_Gravel,14 304 355 339 704 343 1053 345 1403 346 1752 ...
3,002f507.jpg_Sugar,NaN
4,0035ae9.jpg_Fish,NaN
5,0035ae9.jpg_Flower,NaN
6,0035ae9.jpg_Gravel,441 164 606 16 623 10 643 1 776 210 989 26 112...
7,0035ae9.jpg_Sugar,1664 15 1721 7 2007 25 2057 22 2354 31 2396 34...
8,0038327.jpg_Fish,NaN
9,0038327.jpg_Flower,130488 18 130832 27 131178 33 131525 38 131872...
